<a href="https://colab.research.google.com/github/kapoorkhushi03/TRY-IT_ON/blob/main/agnostic%26parse_agnostic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw
import argparse
from tqdm import tqdm

def get_img_agnostic(img, parse, pose_data, w=768, h=1024):
    parse_array = np.array(parse)
    parse_head = ((parse_array == 4).astype(np.float32) +
                  (parse_array == 13).astype(np.float32))
    parse_lower = ((parse_array == 9).astype(np.float32) +
                   (parse_array == 12).astype(np.float32) +
                   (parse_array == 16).astype(np.float32) +
                   (parse_array == 17).astype(np.float32) +
                   (parse_array == 18).astype(np.float32) +
                   (parse_array == 19).astype(np.float32))

    r = 20
    agnostic = img.copy()
    agnostic_draw = ImageDraw.Draw(agnostic)

    length_a = np.linalg.norm(pose_data[5] - pose_data[2])
    length_b = np.linalg.norm(pose_data[12] - pose_data[9])
    point = (pose_data[9] + pose_data[12]) / 2
    pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
    pose_data[12] = point + (pose_data[12] - point) / length_b * length_a

    # mask arms
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 5]], 'gray', width=r*10)
    for i in [2, 5]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')
    for i in [3, 4, 6, 7]:
        if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
            continue
        agnostic_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'gray', width=r*10)
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')

    # mask torso
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*3, pointy-r*6, pointx+r*3, pointy+r*6), 'gray', 'gray')
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 9]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [5, 12]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [9, 12]], 'gray', width=r*12)
    agnostic_draw.polygon([tuple(pose_data[i]) for i in [2, 5, 12, 9]], 'gray', 'gray')

    # mask neck
    pointx, pointy = pose_data[1]
    agnostic_draw.rectangle((pointx-r*7, pointy-r*7, pointx+r*7, pointy+r*7), 'gray', 'gray')
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_lower * 255), 'L'))

    return agnostic


# Modify paths for Google Colab environment
data_path = '/content/drive/MyDrive/data'  # Adjust this path
output_path = '/content/output/output_agnostic'  # Adjust this path

os.makedirs(output_path, exist_ok=True)

for im_name in tqdm(sorted(os.listdir(os.path.join(data_path, 'image')))):
    # Load image
    img_name = im_name
    img = Image.open(os.path.join(data_path, 'image', img_name))

    # Load pose image
    pose_name = im_name.replace('.jpg', '_keypoints.json')

    try:
        with open(os.path.join(data_path, 'openpose_json', pose_name), 'r') as f:
            pose_label = json.load(f)
            pose_data = pose_label['people'][0]['pose_keypoints_2d']
            pose_data = np.array(pose_data)
            pose_data = pose_data.reshape((-1, 3))[:, :2]
    except (IndexError, FileNotFoundError, KeyError) as e:
        print(f"Error processing {pose_name}: {e}")
        continue

    # Load parsing image
    parse_name = im_name.replace('.jpg', '.png')
    parse = Image.open(os.path.join(data_path, 'image-parse-v3', parse_name))

    agnostic = get_img_agnostic(img, parse, pose_data)

    # Save the agnostic image
    output_file = os.path.join(output_path, img_name)
    agnostic.save(output_file)
    print(f"Saved {output_file}")


  1%|▏         | 1/77 [00:00<00:17,  4.45it/s]

Error processing 00000_00_keypoints.json: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/openpose_json/00000_00_keypoints.json'


  3%|▎         | 2/77 [00:00<00:19,  3.86it/s]

Error processing 00001_00_keypoints.json: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/openpose_json/00001_00_keypoints.json'


  4%|▍         | 3/77 [00:00<00:18,  4.04it/s]

...

Error processing 00134_00_keypoints.json: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/openpose_json/00134_00_keypoints.json'


 99%|█████████▊| 76/77 [00:22<00:00,  3.43it/s]

Error processing 00136_00_keypoints.json: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/openpose_json/00136_00_keypoints.json'


100%|██████████| 77/77 [00:22<00:00,  3.40it/s]

Error processing 00137_00_keypoints.json: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/openpose_json/00137_00_keypoints.json'


In [ ]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw
import argparse
from tqdm.notebook import tqdm  # Use tqdm.notebook for Colab's notebook interface

# Mount Google Drive if necessary
from google.colab import drive
drive.mount('/content/drive')

# Define paths
data_path = '/content/drive/My Drive/data'  # Adjust with your Google Drive path
output_path = '/content/drive/My Drive/output/output_agnostic'  # Adjust with your Google Drive path

os.makedirs(output_path, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_img_agnostic(img, parse, pose_data, w=768, h=1024):
    parse_array = np.array(parse)
    parse_head = ((parse_array == 4).astype(np.float32) +
                  (parse_array == 13).astype(np.float32))
    parse_lower = ((parse_array == 9).astype(np.float32) +
                   (parse_array == 12).astype(np.float32) +
                   (parse_array == 16).astype(np.float32) +
                   (parse_array == 17).astype(np.float32) +
                   (parse_array == 18).astype(np.float32) +
                   (parse_array == 19).astype(np.float32))

    r = 20
    agnostic = img.copy()
    agnostic_draw = ImageDraw.Draw(agnostic)

    length_a = np.linalg.norm(pose_data[5] - pose_data[2])
    length_b = np.linalg.norm(pose_data[12] - pose_data[9])
    point = (pose_data[9] + pose_data[12]) / 2
    pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
    pose_data[12] = point + (pose_data[12] - point) / length_b * length_a

    # mask arms
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 5]], 'gray', width=r*10)
    for i in [2, 5]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')
    for i in [3, 4, 6, 7]:
        if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
            continue
        agnostic_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'gray', width=r*10)
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')

    # mask torso
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*3, pointy-r*6, pointx+r*3, pointy+r*6), 'gray', 'gray')
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 9]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [5, 12]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [9, 12]], 'gray', width=r*12)
    agnostic_draw.polygon([tuple(pose_data[i]) for i in [2, 5, 12, 9]], 'gray', 'gray')

    # mask neck
    pointx, pointy = pose_data[1]
    agnostic_draw.rectangle((pointx-r*7, pointy-r*7, pointx+r*7, pointy+r*7), 'gray', 'gray')
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_lower * 255), 'L'))

    return agnostic


In [ ]:
# Iterate through images in the data directory
for im_name in tqdm(os.listdir(os.path.join(data_path, 'image'))):
    # Load image
    img_name = im_name
    img = Image.open(os.path.join(data_path, 'image', img_name))

    # Load pose image
    pose_name = im_name.replace('.jpg', '_keypoints.json')
    try:
        with open(os.path.join(data_path, 'openpose_json', pose_name), 'r') as f:
            pose_label = json.load(f)
            pose_data = pose_label['people'][0]['pose_keypoints_2d']
            pose_data = np.array(pose_data)
            pose_data = pose_data.reshape((-1, 3))[:, :2]
    except (IndexError, FileNotFoundError, KeyError) as e:
        print(f"Error processing {pose_name}: {e}")
        continue

    # Load parsing image
    parse_name = im_name.replace('.jpg', '.png')
    parse = Image.open(os.path.join(data_path, 'image-parse-v3', parse_name))

    # Generate agnostic image
    agnostic = get_img_agnostic(img, parse, pose_data)

    # Save the agnostic image
    agnostic_file = os.path.join(output_path, img_name)
    agnostic.save(agnostic_file)
    print(f"Saved agnostic image: {agnostic_file}")

    # Optionally, save the parse image as well
    parse_file = os.path.join(output_path, parse_name)
    parse.save(parse_file)
    print(f"Saved parse image: {parse_file}")


  0%|          | 0/103 [00:00<?, ?it/s]

Saved agnostic image: /content/drive/My Drive/output/output_agnostic/00101_00.jpg
Saved parse image: /content/drive/My Drive/output/output_agnostic/00101_00.png
Saved agnostic image: /content/drive/My Drive/output/output_agnostic/00103_00.jpg
Saved parse image: /content/drive/My Drive/output/output_agnostic/00103_00.png
Saved agnostic image: /content/drive/My Drive/output/output_agnostic/00011_00.jpg
Saved parse image: /content/drive/My Drive/output/output_agnostic/00011_00.png
Saved agnostic image: /content/drive/My Drive/output/output_agnostic/00048_00.jpg
Saved parse image: /content/drive/My Drive/output/output_agnostic/00048_00.png
Saved agnostic image: /content/drive/My Drive/output/output_agnostic/00003_00.jpg
Saved parse image: /content/drive/My Drive/output/output_agnostic/00003_00.png
Saved agnostic image: /content/drive/My Drive/output/output_agnostic/00044_00.jpg
Saved parse image: /content/drive/My Drive/output/output_agnostic/00044_00.png
Saved agnostic image: /content/dri